In [ ]:
# ========================================
# Cell 1: 라이브러리 import 및 설정
# ========================================

import os
import sys
import getpass
import shutil
from datetime import datetime
sys.path.append(os.path.abspath('../src'))

from config import *
from collectors.aem_collector import AEMCollector
from collectors.pdf_collector import PDFCollector
from collectors.package_builder import create_package

# 필요한 디렉토리들 생성
os.makedirs(AEM_BATCHES_DONE_DIR, exist_ok=True)
os.makedirs(PDF_BATCHES_DONE_DIR, exist_ok=True)

# 지원할 언어 쌍들 정의
LANGUAGE_PAIRS = [
    ('en', 'ko'),  # 영어-한국어
    ('en', 'ja'),  # 영어-일본어
]

print(f"🌐 지원 언어 쌍: {LANGUAGE_PAIRS}")
print(f"📁 작업 디렉토리 확인:")
print(f"   - AEM TODO: {AEM_BATCHES_TODO_DIR}")
print(f"   - AEM DONE: {AEM_BATCHES_DONE_DIR}")
print(f"   - PDF TODO: {PDF_BATCHES_TODO_DIR}")
print(f"   - PDF DONE: {PDF_BATCHES_DONE_DIR}")




In [ ]:
# ========================================
# Cell 2: AEM 인증 정보 입력
# ========================================

print("🔐 AEM 접속 정보를 입력하세요.")
username = input("사용자 이름: ")
password = getpass.getpass("비밀번호: ")




In [ ]:
# ========================================
# Cell 3: AEM 배치 처리 (기존과 동일)
# ========================================

all_collected_files = []

print("\n" + "="*80)
print("--- AEM 페이지 배치 처리를 시작합니다. ---")

# AEM 배치 파일 존재 확인
if not os.path.exists(AEM_BATCHES_TODO_DIR):
    print(f"❌ AEM TODO 디렉토리를 찾을 수 없습니다: {AEM_BATCHES_TODO_DIR}")
elif not os.listdir(AEM_BATCHES_TODO_DIR):
    print("⚠️ 처리할 AEM 배치 파일이 없습니다. 2_setup_batches.ipynb를 먼저 실행하세요.")
else:
    while True:
        batch_files = sorted([f for f in os.listdir(AEM_BATCHES_TODO_DIR) if f.endswith('.list')])
        if not batch_files:
            print("\n✅ 모든 AEM 배치가 처리되었습니다.")
            break
        
        current_batch_file = batch_files[0]
        current_batch_path = os.path.join(AEM_BATCHES_TODO_DIR, current_batch_file)
        
        print(f"\n🤖 다음 AEM 배치를 처리합니다: {current_batch_file}")
        
        with open(current_batch_path, 'r', encoding='utf-8') as f:
            page_paths = [line.strip() for line in f if line.strip()]
            
        if not page_paths:
            print("   - ✅ 배치 파일이 비어있어 건너뜁니다.")
            shutil.move(current_batch_path, os.path.join(AEM_BATCHES_DONE_DIR, current_batch_file))
            continue

        try:
            collector = AEMCollector(username, password)
            collected_files = collector.collect_snapshots_for_batch(page_paths)
            all_collected_files.extend(collected_files)
            
            print(f"   - ✅ 배치 성공: {len(collected_files)}개 파일 수집.")
            shutil.move(current_batch_path, os.path.join(AEM_BATCHES_DONE_DIR, current_batch_file))
            
        except Exception as e:
            print(f"   - ❌ 배치 실패: {current_batch_file}. 오류: {e}")
            break




In [ ]:
# ========================================
# Cell 4: 다국어 PDF 배치 처리
# ========================================

all_collected_files = []

print("\n" + "="*80)
print("--- 다국어 PDF 배치 처리를 시작합니다. ---")

# PDF 배치 파일 존재 확인
if not os.path.exists(PDF_BATCHES_TODO_DIR):
    print(f"❌ PDF TODO 디렉토리를 찾을 수 없습니다: {PDF_BATCHES_TODO_DIR}")
elif not os.listdir(PDF_BATCHES_TODO_DIR):
    print("⚠️ 처리할 PDF 배치 파일이 없습니다. 2_setup_batches.ipynb를 먼저 실행하세요.")
else:
    for source_lang, target_lang in LANGUAGE_PAIRS:
        print(f"\n🌐 {source_lang.upper()}-{target_lang.upper()} PDF 처리 중...")
        
        # 해당 언어 쌍의 배치 파일들 찾기
        language_suffix = f"{source_lang}_{target_lang}"
        batch_pattern = f"pdf_batch_{language_suffix}_"
        
        # 해당 언어 쌍의 배치 파일 확인
        available_batches = [
            f for f in os.listdir(PDF_BATCHES_TODO_DIR) 
            if f.startswith(batch_pattern) and f.endswith('.csv')
        ]
        
        if not available_batches:
            print(f"   - ⚠️ {source_lang.upper()}-{target_lang.upper()} 배치 파일을 찾을 수 없습니다.")
            continue
        
        print(f"   - 📋 {len(available_batches)}개의 배치 파일 발견")
        
        while True:
            batch_files = sorted([
                f for f in os.listdir(PDF_BATCHES_TODO_DIR) 
                if f.startswith(batch_pattern) and f.endswith('.csv')
            ])
            
            if not batch_files:
                print(f"   - ✅ {source_lang.upper()}-{target_lang.upper()} 모든 배치 처리 완료.")
                break

            current_batch_file = batch_files[0]
            current_batch_path = os.path.join(PDF_BATCHES_TODO_DIR, current_batch_file)
            
            print(f"\n📖 처리 중: {current_batch_file}")
            
            try:
                collector = PDFCollector(username, password)
                collected_files = collector.collect_pdfs_for_batch(
                    current_batch_path, 
                    language_pair=(source_lang, target_lang)
                )
                all_collected_files.extend(collected_files)
                
                print(f"   - ✅ 배치 성공: {len(collected_files)}개 파일 수집.")
                shutil.move(current_batch_path, os.path.join(PDF_BATCHES_DONE_DIR, current_batch_file))
                
            except Exception as e:
                print(f"   - ❌ 배치 실패: {current_batch_file}. 오류: {e}")
                break

print(f"\n🎉 전체 PDF 수집 완료! 총 {len(all_collected_files)}개 파일")




In [ ]:
# ========================================
# Cell 5: 최종 패키징
# ========================================

if all_collected_files:
    # 언어별 파일 분류 요약
    file_by_type = {}
    for file_info in all_collected_files:
        file_type = file_info.file_type
        if file_type not in file_by_type:
            file_by_type[file_type] = 0
        file_by_type[file_type] += 1
    
    print(f"\n📊 수집된 파일 요약:")
    for file_type, count in file_by_type.items():
        print(f"   - {file_type}: {count}개")
    
    # 패키지 생성
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    package_name = f"multilingual_data_package_{timestamp}"
    create_package(package_name, all_collected_files)
    
else:
    print("\n⚠️ 수집된 파일이 없어 패키지를 생성하지 않습니다.")

print("\n🎉 모든 다국어 데이터 수집 및 패키징 작업이 완료되었습니다!")